Two approaches
1) scraping existing website and store all drug names - class names
2)  api

sample test cases
name -- pharmocologic class

1)Betadine → Antiseptic (Povidone-Iodine)

2)Naproxen → Nonsteroidal Anti-Inflammatory Drug (NSAID)

3)oisturizing Antibacterial → Antibacterial agent (Exact classification depends on active ingredient)

4)NonExistentDrug - No data found for the drug

## using rxcui - cannot get class name


In [1]:
# import requests
# import json
# # def get_rxcui(drug_name):
# #     """Fetch the RxCUI (RxNorm Concept Unique Identifier) for a given drug name."""
# #     url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={drug_name}&search=1"
# #     response = requests.get(url)
# #     data = response.json()

# #     if 'idGroup' in data and 'rxnormId' in data['idGroup']:
# #         return data['idGroup']['rxnormId'][0]
# #     return None

# def get_pharmacologic_class(drug_name):

#     """Retrieve the pharmacologic class for a given RxCUI."""
#     url = f"https://rxnav.nlm.nih.gov/REST/rxclass/class/byDrugName.json?drugName={drug_name}&relaSource=MEDRT&relas=may_treat"
#     response = requests.get(url)
#     print(f"response is:{response}")
#     data = response.json()
#     print(json.dumps(data,indent=4))

#     if 'propConceptGroup' in data and 'propConcept' in data['propConceptGroup']:
#         return [concept['propValue'] for concept in data['propConceptGroup']['propConcept']]
#     return None

# def main():
#     drug_name = input("Enter the drug name: ").strip()
#     rxcui = get_pharmacologic_class(drug_name)

#     if not rxcui:
#         print("Drug not found in RxNorm database.")
#         return

#     pharmacologic_classes = get_pharmacologic_class(rxcui)

#     if pharmacologic_classes:
#         print(f"Pharmacologic Class for {drug_name}: {', '.join(pharmacologic_classes)}")
#     else:
#         print(f"No pharmacologic class information found for {drug_name}.")

# if __name__ == "__main__":
#     main()


response is:<Response [200]>
{}
Drug not found in RxNorm database.


## drug bank api -paid


In [2]:
# base_url='https://api.drugbank.com/v1/us'
# response=requests.get(base_url)
# if response.status_code==200:
#   try:
#     # data=response.json()
#     print(response.text)
#     print(response.json())
#   except requests.exceptions.JSONDecodeError as e:
#     print(f"Error decoding JSON: {e}")
# else:
#   print(f"Error: {response.status_code}")




Error: 401


## using open fda -

In [57]:
import requests
import json
def get_pharmacological_class(drug_name):
    # OpenFDA API endpoint
    base_url = "https://api.fda.gov/drug/label.json"
    search_url = f"{base_url}?search=openfda.brand_name:{drug_name}"

    try:
        response = requests.get(search_url)
        response.raise_for_status()
        search_data = response.json()


        if not search_data.get('results', []):
            return f"Error: No results found for '{drug_name}' in OpenFDA."


        label = search_data['results'][0]
        openfda = label.get('openfda', {})

        # Extract pharmacological class information
        pharm_class_moa = openfda.get('pharm_class_moa', [])
        pharm_class_pe = openfda.get('pharm_class_pe', [])
        pharm_class_cs = openfda.get('pharm_class_cs', [])
        pharm_class_epc = openfda.get('pharm_class_epc', [])

        # Combine all pharmacological class information
        pharmacological_class = {
            "Mechanism of Action (MoA)": pharm_class_moa,
            "Pharmacologic Effect (PE)": pharm_class_pe,
            "Chemical Structure (CS)": pharm_class_cs,
            "Established Pharmacologic Class (EPC)": pharm_class_epc
        }

        if any(pharmacological_class.values()):
            return pharmacological_class
        else:
            return f"Pharmacological class not found for '{drug_name}' in OpenFDA."

    except requests.exceptions.RequestException as e:
        return f"Error: Failed to fetch data from OpenFDA. Details: {e}"
    except ValueError as e:
        return f"Error: Invalid JSON response. Details: {e}"

# Example usage
drug_name=input("Enter the drug name: ").strip()
drug_name=drug_name.lower()
pharm_class = get_pharmacological_class(drug_name)
json.dumps(pharm_class)
# print(f"Pharmacological class of {drug_name}: {pharm_class}")

'"Pharmacological class not found for \'amoxicillin\' in OpenFDA."'

In [56]:
print("Given drug name:",drug_name)
if (type(pharm_class)==str and "Error" in pharm_class):
    print("not present in openFDA")
else:
    print(f"Generic Pharmacological class name is:")
    for x in pharm_class['Established Pharmacologic Class (EPC)']:
        print(x)

Given drug name: isopropyl alcohol
Generic Pharmacological class name is:
Nonsteroidal Anti-inflammatory Drug [EPC]
